In [1]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Convolution2D
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization

from tensorflow.keras import Model

from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import initializers
import tensorflow.keras.backend as K

print(tf.__version__)

2.0.0-alpha0


In [2]:
mnist = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
# x_train = x_train[..., tf.newaxis]
# x_test = x_test[..., tf.newaxis]

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [4]:
x_train.shape

(50000, 32, 32, 3)

In [5]:
class DenseNet(Model):
    def __init__(self, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, 
                 dropout_rate=0.0, weight_decay=1e-4, classes=1000, batch_size=32):
        super(DenseNet, self).__init__()

        self.eps = 1.1e-5
                
        # compute compression factor
        compression = 1.0 - reduction
        
        self.concat_axis = 3

        # From architecture for ImageNet (Table 1 in the paper)
        nb_filter = 64
        nb_layers = [6,12,24,16] # For DenseNet-121

        self.initial_layers = []
        self.initial_layers.append(ZeroPadding2D((3, 3), name='conv1_zeropadding', 
                                   input_shape=(32, 32, 3), batch_size=batch_size))
        self.initial_layers.append(Convolution2D(nb_filter, 7, 2, name='conv1', use_bias=False))
        self.initial_layers.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name='conv1_bn'))
        self.initial_layers.append(Activation('relu', name='relu1'))
        self.initial_layers.append(ZeroPadding2D((1, 1), name='pool1_zeropadding'))
        self.initial_layers.append(MaxPooling2D((3, 3), strides=(2, 2), name='pool1'))
        
        self.dense_blocks = []
        self.transition_blocks = []

        # Add dense blocks
        for block_idx in range(nb_dense_block - 1):
            stage = block_idx+2
            block, nb_filter = self.dense_block(stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
            self.dense_blocks.append(block)
            
            # Add transition_block
            self.transition_blocks.append(self.transition_block(stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay))
            nb_filter = int(nb_filter * compression)

        final_stage = stage + 1
        block, nb_filter = self.dense_block(final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
        self.dense_blocks.append(block)
        
        self.final_layers = self.final_block(nb_filter, classes)       
    
    def final_block(self, nb_filter, classes):
        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name='conv_final_blk_bn'))
        block.append(Activation('relu', name='relu_final_blk'))
        block.append(GlobalAveragePooling2D(name='pool_final'))
        block.append(Dense(classes, name='fc6'))
        block.append(Activation('softmax', name='prob'))
        return block
    
    def conv_block(self, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
        conv_name_base = 'conv' + str(stage) + '_' + str(branch)
        relu_name_base = 'relu' + str(stage) + '_' + str(branch)

        # 1x1 Convolution (Bottleneck layer)
        inter_channel = nb_filter * 4  
        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_x1_bn'))
        block.append(Activation('relu', name=relu_name_base+'_x1'))
        block.append(Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))

        # 3x3 Convolution
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_x2_bn'))
        block.append(Activation('relu', name=relu_name_base+'_x2'))
        block.append(ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding'))
        block.append(Convolution2D(nb_filter, 3, 1, name=conv_name_base+'_x2', use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))
        return block
                                        

    def dense_block(self, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, 
                    grow_nb_filters=True):
        block = []
        for i in range(nb_layers):
            branch = i+1
            block.append(self.conv_block(stage, branch, growth_rate, dropout_rate, weight_decay))

            if grow_nb_filters:
                nb_filter += growth_rate

        return block, nb_filter
                                        
    def transition_block(self, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
        conv_name_base = 'conv' + str(stage) + '_blk'
        relu_name_base = 'relu' + str(stage) + '_blk'
        pool_name_base = 'pool' + str(stage) 

        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_bn'))
        block.append(Activation('relu', name=relu_name_base))
        block.append(Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))

        block.append(AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base))

        return block

    def call(self, inputs):
        x = inputs
        for layer in self.initial_layers:
            x = layer(x)
        i = 0
        for transition_block in self.transition_blocks:
            concat_feat = x
            for conv_block in self.dense_blocks[i]:                                    
                for layer in conv_block:                                    
                    x = layer(x)
                x = tf.concat([concat_feat, x], self.concat_axis)
                concat_feat = x
            for layer in transition_block:                                    
                x = layer(x)
            i += 1
        concat_feat = x
        for conv_block in self.dense_blocks[i]:                                    
            for layer in conv_block:                                    
                x = layer(x)
            x = tf.concat([concat_feat, x], self.concat_axis)
            concat_feat = x
        for layer in self.final_layers:
            x = layer(x)                                      
        return x

model = DenseNet(classes=10)
# model.load_weights(weights_path) 

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(tf.cast(images, tf.float32))
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


In [9]:
@tf.function
def test_step(images, labels):
    predictions = model(tf.cast(images, tf.float32))
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    step = 0
    for images, labels in train_ds:
        train_step(images, labels)

        if (step%5 == 0):
            template = 'Step {}, Train_Step_Loss: {}'
            print (template.format(step, train_loss.result()))
        step += 1

    step = 0
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

        if (step%5== 0):
            template = 'Step {}, Test_Step_Loss: {}'
            print (template.format(step, test_loss.result()))
        step += 1

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))


Step 0, Train_Step_Loss: 2.2992844581604004
Step 5, Train_Step_Loss: 2.3177640438079834
